In [2]:
import numpy as np
import random
import pandas as pd
from scipy.optimize import minimize
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io as sio
import statsmodels.api as sm

# Important Note

### purpose of this file is to get parameters (alphaG, alphaL, beta) of Q learning for real behavioral data

### reward in this file is always subjective reward, i.e. 0 <==> the rat did not get reward

## Load Data

In [4]:
day1 = pd.read_csv('Data/Day 1.csvQ_Analysis.csv')
day21 = pd.read_csv('Data/Day 21.csvQ_Analysis.csv')

In [5]:
day1.columns

Index(['Unnamed: 0', 'subject', 'prog', 'target_lat', 'non_target_lat', 'ITI',
       'ISI', 'Tone', 'TargetOutcome', 'NonTargetOutcome', 'NAN', 'TargetSide',
       'Omission', 'TimeOut', 'SessionTime', 'NAN.1', 'NAN.2',
       'SwitchContingency', 'trial', 'position'],
      dtype='object')

In [6]:
# Let rat 30 be chosen
# low tone <==> reward
day1_30 = day1[day1['subject']==30]
day21_30 = day21[day21['subject']==30]

In [7]:
day1_30 = day1_30[['Tone','TargetOutcome','NonTargetOutcome','TargetSide']]
day21_30 = day21_30[['Tone','TargetOutcome','NonTargetOutcome','TargetSide']]

In [8]:
day1_30[:10]

,Tone,TargetOutcome,NonTargetOutcome,TargetSide
291,2000.0,1.0,0.0,1.0
292,5000.0,0.0,-1.0,1.0
293,5000.0,0.0,-1.0,1.0
294,2000.0,0.0,1.0,1.0
295,5000.0,-1.0,0.0,1.0
296,5000.0,0.0,-1.0,1.0
297,2000.0,0.0,1.0,1.0
298,5000.0,0.0,-1.0,1.0
299,5000.0,0.0,-1.0,1.0
300,5000.0,0.0,-1.0,1.0


## Get Actions and Rewards

In [9]:
#df: 4 columns of 'Tone','TargetOutcome','NonTargetOutcome','TargetSide'; df is for one rat in one session
# consisting of 1(left) and 2(right)
# suppose TargetSide=1.0 means left, 4.0 means right
def get_action(df):
    actions = []
    targetSide = df['TargetSide'].tolist()
    targetOut = df['TargetOutcome'].tolist()
    for i in range(df.shape[0]):
        act = -1
        if(targetSide[i]==1.0):
            if(targetOut[i]==0):
                act = 2
            else:
                act = 1
        else:
            if(targetOut[i]==0):
                act = 1
            else:
                act = 2
        actions.append(act)
        
    return actions
    

In [10]:
# df: 4 columns of 'Tone','TargetOutcome','NonTargetOutcome','TargetSide'; df is for one rat in one session
# reward is subjective, depending on whether the rat got reward
def get_reward(df):
    rewards = []
    targetOut = df['TargetOutcome'].tolist()
    nontargetOut = df['NonTargetOutcome'].tolist()
    
    for i in range(df.shape[0]):
        rew = -1
        if(targetOut[i] == 1 or nontargetOut[i] == 1):
            rew = 1
        else:
            rew = 0
        rewards.append(rew)
        
    return rewards

## MLE Functions

In [11]:
# action is from choice_Log consisting of 1 and 2s
# reward is subjective, depending on whether the rat got reward
def neg_log_likelihood_2Q_2alpha(alphaG,alphaL,beta,actions,rewards,Q=[0.1,0.1],gamma=0,Qleft=True): 
    n = len(actions)
    sum_ll = 0
    for i in range(n):
        turn = int(actions[i] - 1)
        rew = int(rewards[i])
        
        if int(rew) == 1: # alpha_gain
            Q[turn] = Q[turn] + alphaG*(rew - Q[turn] + gamma*np.max(Q))
        elif int(rew) == 0: # alpha_loss
            Q[turn] = Q[turn] + alphaL*(rew - Q[turn] + gamma*np.max(Q))
        
        temp = 1/(np.exp(0-beta*(Q[0])) + 1)
        
        if int(turn) == 0:
            prob = temp
        else:
            prob = 1 - temp
        
        sum_ll = sum_ll - np.log(prob + np.exp(0-8)) # add a smoother to avoid warnings
    
    return sum_ll/50 #rescale

In [12]:
# params = [alphaG0,alphaL0,beta0]
# args = [actions,rewards]
def helper_func_2alpha(params,args):
    alphaG0 = params[0]
    alphaL0 = params[1]
    beta0 = params[2]
    actions = args[0]
    rewards = args[1]
    
    sum_ll = neg_log_likelihood_2Q_2alpha(alphaG0,alphaL0,beta0,actions,rewards)
    
    return sum_ll

In [13]:
# function that estimates the maximum-likelihood beta_hat numerically
# parameters: actions, a numpy array recording action of agent in each turn; beta is the parameter in density func
# return minimization summary and print beta_hat
def MLE_2alpha(actions,rewards,alphaG0,alphaL0,beta0):
    initial_guess = [alphaG0,alphaL0,beta0]
    args = [actions,rewards]
    bounds = ((0,1),(0,1),(0,20))
    result = minimize(helper_func_2alpha,initial_guess,args=args,bounds = bounds)
    if(result.success):
        #print(result.message)
        #print('The MLE for beta is', result.x)
        #print('Iteration =', result.nit)
        a=0
    else:
        print('The optimization did not converge, beta0 equals', beta0,', and alphaG0 equals',alphaG0)
    return result
    
    

In [17]:
def my_minimize(actions,rewards, alphaG=0, alphaL=0, beta=0):
    likelihoods = []
    #step = 1/10  #1/int(iteration**(1/3))
    alphaG_arr = np.arange(0,1,0.05)
    alphaL_arr = np.arange(0,1,0.05)
    beta_arr = np.arange(0,20,0.5)
    num_G = len(alphaG_arr)
    num_L = len(alphaL_arr)
    num_b = len(beta_arr)
    
    index_G = 0
    index_L = 0
    index_b = 0
    
    temp = 100000
    
    for i in range(num_G):
        for j in range(num_L):
            for k in range(num_b):
                
                ll = neg_log_likelihood_2Q_2alpha(alphaG_arr[i],alphaL_arr[j],beta_arr[k],actions,rewards)
                likelihoods.append(ll)
                
                if(ll < temp):
                    temp = ll
                    index_G = i
                    index_L = j
                    index_b = k
    
    alphaG_ = alphaG_arr[index_G]
    alphaL_ = alphaL_arr[index_L]
    beta_ = beta_arr[index_b]
    
    return [alphaG_,alphaL_,beta_]

In [ ]:
def fast_MLE(actions,rewards, alphaG=0, alphaL=0, beta=0):
    likelihoods = []
    #step = 1/10  #1/int(iteration**(1/3))
    alphaG_arr = np.arange(0,1,0.05)
    alphaL_arr = np.arange(0,1,0.05)
    beta_arr = np.arange(0,20,0.5)
    num_G = len(alphaG_arr)
    num_L = len(alphaL_arr)
    num_b = len(beta_arr)
    
    index_G = 0
    index_L = 0
    index_b = 0
    
    temp = 100000
    
    for i in range(num_G):
        for j in range(num_L):
            for k in range(num_b):
                
                ll = neg_log_likelihood_2Q_2alpha(alphaG_arr[i],alphaL_arr[j],beta_arr[k],actions,rewards)
                likelihoods.append(ll)
                
                if(ll < temp):
                    temp = ll
                    index_G = i
                    index_L = j
                    index_b = k
    
    alphaG_ = alphaG_arr[index_G]
    alphaL_ = alphaL_arr[index_L]
    beta_ = beta_arr[index_b]
    
    return [alphaG_,alphaL_,beta_]